지금까지 살펴본 신경망은 피드포워드 유형의 신경망이다. 피드포워드란 흐름이 단방향을 신경망을 말한다.

그러나 피드포워드 신경망은 **시계열 데이터의 성질(패턴)을 충분히 학습할 수 없다**는 커다란 단점이 있다.

그래서 **순환 신경망 <sup>Recurrent Neural Network</sup> (RNN)** 이 등장하게 되었다.

# 확률과 언어 모델

## word2vec을 확률 관점에서 바라보다

먼저 word2vec의 CBOW 모델을 간단히 복습해보자. 

말뭉치 $w_1, w_2, ...,w_{t-1},w_{t},w_{t+1},,..., w_T$ 에서 $w_{t-1}$과 $w_{t+1}$이 주어졌을 때 타깃 $w_t$가 될 확률은 다음과 같다. 

$$P(w_t|w_{t-1},w_{t+1})$$

이번에는 맥락을 왼쪽 윈도우만으로 한정해보자.

<img src="../imgs/fig 5-2.png" width="400" align='center'>

이제 확률 식은 다음으로 같다.

$$P(w_t|w_{t-2},w_{t-1})$$

이전에 나온 단어들의 확률 값으로 이후 단어를 예측하는 것. 이 식으로부터 언어 모델이 등장한다.

## 언어 모델

언어 모델 <sup>Language Model</sup>은 **특정한 단어의 시퀀스에 대해서, 그 시퀀스가 일어날 가능성이 어느 정도인지 (얼마나 자연스러운 단어 순서인지)를 확률로 평가한다.**

    - "you say goodbye" : 높은 확률
    - "you say good die" : 낮은 확률

응용 분야 

- 기계 번역과 음성 인식에서 문장이 얼마나 자연스러운지 판단하여 더 높은 자연스러움을 가진 문장을 반환
- 단어 순서의 자연스러움을 토대로 새로운 문장을 생성

---

#### 이제 언어 모델을 수식으로 이해해보자

$w_1,...,w_m$ 이라는 m개의 단어로 된 문장을 생각해보자.

이때 단어가 $w_1,...,w_m$이라는 순서로 출현할 확률을 $P(w_1,...,w_m)$의 동시 확률로 나타낼 수 있다.

이 동시 확률은 다음과 같이 분해하여 쓸 수 있다. 

<img src="../imgs/e 5-4.png" width="400" align='center'>

동시 확률 $P(w_1,...,w_m)$는 사후 확률의 총 곱인 $\prod{P(w_t|w_1,...,w_{t-1})}$ 으로 대표될 수 있다.    
여기서의 사후 확률은 **타깃 단어보다 왼쪽에 있는 모든 단어**를 맥락으로 했을 때의 확률과 같다. 

즉, 단어가 순서대로 출현할 확률은 동시 확률로 나타내질 수 있고, 이 동시 확률은 사후 확률의 총 곱으로 나타내질 수 있으니까. 우리의 목표는 사후 확률 $P(w_t|w_1,...,w_{t-1})$를 구하는 것!

## CBOW 모델을 언어 모델로?

그렇다면 word2vec의 CBOW 모델을 억지로 언어 모델에 적용하려면??

-> 맥락의 크기를 특정 값으로 한정하여 근사적으로 나타낼 수 있다! 맥락의 크기를 왼쪽 2개 단어로 한정한다면, 다음과 같이 표현 가능

$P(w_1,...,w_m) = \prod_{t=1}^{m}{P(w_t|w_1,...,w_{t-1})}\approx \prod_{t=1}^{m}{P(w_t|w_{t-2},w_{t-1})}$

맥락의 길이는 5나 10으로 임의로 설정 가능. 어쨌거나 **맥락의 크기는 특정 크기로 고정**된다.

#### 한계점

    이처럼 맥락의 크기가 고정될 경우 아래와 같이 맥락 크기보다 앞에 단서 단어가 나오는 케이스는 해결하기 힘들다.

 `Tom` was watching TV in his room. Mary came into the room. Mary said hi to `?`
    
   **1) 그렇다면 맥락의 크기를 20이나 30으로 키우면 되지 않을까?**

    but CBOW 모델에서는 레이어 내부에서 단어 벡터들의 합계를 내는 과정에서 맥락 안의 순서가 무시되기 때문에 적합하지 않다.

   **2) 엥 그렇다면 단어 벡터들 합하지 말고 concatenate하면 되지 않을까??**
    
    맥락의 크기에 비례해 매개변수가 증가한다. 

그러나 RNN의 단점도 많은 것으로 아는데...

# RNN이란

